In [8]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
examples = torch.load('../condqa_old/data/dev_examples', map_location='cpu')

In [36]:
print([i['has_answer'] for i in examples[25]['document'] if i['has_answer'] != -1])


[3, 0, 1, 2, 0, 1, 2]


In [45]:
from utils import convert_examples_to_inputs, Tokenizer
tokenizer = Tokenizer('../condqa_old/model')
inputs = convert_examples_to_inputs(examples, tokenizer)

converting examples to inputs..: 100%|██████████| 352/352 [00:02<00:00, 132.75it/s]


In [55]:
inputs[38][-2]

tensor(29)

In [59]:
inputs[39][-1].nonzero()


tensor([[ 969,    0,    1],
        [1142,    0,    0]])